In [29]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.core.os_manager import ChromeType
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import sys

In [30]:
# Initialize the browser
if sys.platform == "win32":
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
else:
    driver = webdriver.Chrome(service=Service(ChromeDriverManager(chrome_type=ChromeType.CHROMIUM).install()))

driver.maximize_window()
driver.get('https://www.linkedin.com/')


wait = WebDriverWait(driver, 10)

In [31]:
# Function to smoothly scroll to the bottom of the page
def smooth_scroll_to_bottom():
    # Get the initial page height
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
        # Scroll down by a small increment
        driver.execute_script("window.scrollBy(0, document.body.scrollHeight);")
        
        # Wait for new content to load
        time.sleep(0.5)  # Adjust the sleep time as needed
        
        # Get the new page height
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        # If no new content is loaded, stop scrolling
        if new_height == last_height:
            break
        
        # Update the last height
        last_height = new_height


In [32]:

# Navigate to the jobs page 
wait.until(EC.presence_of_element_located((By.CLASS_NAME, "nav")))
driver.find_element(By.XPATH, "//a[@href='https://www.linkedin.com/jobs/search?trk=guest_homepage-basic_guest_nav_menu_jobs']").click()

# close if there is any popup modal 
try:
    driver.find_element(By.CLASS_NAME, "modal__dismiss").click()
except Exception as e:
    print(f"Model {e}")
    


In [33]:
# Scroll to load all content
smooth_scroll_to_bottom()
# job_search_results = driver.find_element(By.CLASS_NAME, "jobs-search__results-list")
jobs = driver.find_elements(By.CLASS_NAME, "job-search-card")
print(len(jobs))

60


In [ ]:
# locate the list of jobs 
jobs = driver.find_elements(By.CLASS_NAME, "job-search-card")
print(len(jobs))
jobs_fetched = []
number_of_jobs_to_be_scrap = 1
try:
    # loop through each job
    for job_counter in range(number_of_jobs_to_be_scrap):  
        # re-locate the list of jobs of to avoid stale elements
        jobs = driver.find_elements(By.CLASS_NAME, "job-search-card")
        number_of_jobs_loaded = len(jobs)
        
        # click on the current job 
        job = jobs[job_counter]
        job.find_element(By.CLASS_NAME, "base-card__full-link").click()
    
        # wait for the detailed page to load 
        time.sleep(1)
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, "two-pane-serp-page__detail-view")))
    
        # Extract detailed information 
        job_title = driver.find_element(By.CLASS_NAME, "top-card-layout__title").text
        company_name = driver.find_element(By.CLASS_NAME, "topcard__org-name-link").text
        job_location = driver.find_element(By.CLASS_NAME, "topcard__flavor--bullet").text
        job_description= driver.find_element(By.CLASS_NAME, "decorated-job-posting__details").get_attribute("innerText")

        job_detail = {
            "Title": job_title,
            "Company": company_name,
            "Location": job_location,
            "Description": job_description
        }
        jobs_fetched.append(job_detail)
        
        print("-" * 100)
        print(f'Job {job_counter+1}')
        print("-" * 100)
        print(f"Title: {job_title}")
        print(f"Company: {company_name}")
        print(f"Location: {job_location}")
        print(f"Description: {job_description}")
        print("-" * 100)
    
        # Go back to the list of jobs 
        driver.back
        
        try:
            if job_counter == number_of_jobs_loaded-1:
                time.sleep(2)
                try:
                    see_more_jobs = driver.find_element(By.XPATH, "//button[@aria-label='See more jobs']")
                    if see_more_jobs:
                        see_more_jobs.click()
                        time.sleep(2)
                except:
                    pass
                    
                finally:
                    # smooth_scroll_to_bottom()
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            try:
                see_more_jobs = driver.find_element(By.XPATH, "//button[@aria-label='See more jobs']")
                if see_more_jobs:
                    time.sleep(2)
                    # see_more_jobs.click()
                    driver.execute_script("arguments[0].click();", see_more_jobs)
                    # jobs = driver.find_elements(By.CLASS_NAME, "job-search-card")
            except:
                pass       

        except:
            pass
    
        
    
        # # wait for the list to load again 
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, "job-search-card")))
    
except Exception as e:
                print(f'Scraped {job_counter} jobs, there are no more jobs in the search results: {e}')

finally: 
    print(f'Scraped {len(jobs_fetched)} jobs sucessfully')
    # close the brower
    driver.quit()


60
----------------------------------------------------------------------------------------------------
Job 1
----------------------------------------------------------------------------------------------------
Title: Escritor de Blogs
Company: Outlier
Location: Vermont, United States
Description: Outlier ayuda a las empresas más innovadoras del mundo a mejorar sus modelos de inteligencia artificial proporcionando retroalimentación humana. ¿Eres un redactor experimentado en español y te gustaría usar tu experiencia para entrenar modelos de inteligencia artificial?

Sobre la oportunidad: Outlier está buscando redactores talentosos con fluidez en español para ayudar a entrenar modelos de inteligencia artificial generativa. Esta oportunidad freelance es remota y ofrece horarios flexibles, por lo que puedes trabajar cuando mejor te convenga.

Cómo puedes contribuir

Leer textos en español (Estados Unidos) para clasificar una serie de respuestas generadas por un modelo de IA. Escribir una h

In [35]:
len(jobs_fetched)

100

In [36]:
company_url = driver.find_element(By.XPATH, "//a[@href]")

MaxRetryError: HTTPConnectionPool(host='localhost', port=56437): Max retries exceeded with url: /session/5a8f76fceeabf6f2b53dc935583bfba6/element (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000021A3F07D8C0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [ ]:
company_url.get_attribute("href")

'https://www.linkedin.com/jobs/search?trk=guest_homepage-basic_guest_nav_menu_jobs&currentJobId=4202502358&position=10&pageNum=0#main-content'